In [ ]:
import pandas as pd
import numpy as np
import math
import datetime as dt
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.preprocessing import MinMaxScaler

!pip install tensorflow
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, GRU

from itertools import cycle

! pip install plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Data/RELIANCE.csv.xls')
df.head(10)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2020-08-19,2141.000000,2154.000000,2121.350098,2131.550049,2124.715088,15731396.0
1,2020-08-20,2120.000000,2123.899902,2088.000000,2097.050049,2090.325684,10401212.0
2,2020-08-21,2118.000000,2122.000000,2077.000000,2081.850098,2075.174316,11667129.0
3,2020-08-24,2091.399902,2104.500000,2070.500000,2095.750000,2089.029785,15098991.0
4,2020-08-25,2106.000000,2111.300049,2078.000000,2082.100098,2075.423584,8947563.0
5,2020-08-26,2085.000000,2147.949951,2047.000000,2137.300049,2130.446533,27630190.0
6,2020-08-27,2148.000000,2150.000000,2103.250000,2110.600098,2103.832275,12961252.0
7,2020-08-28,2120.000000,2132.899902,2106.250000,2116.149902,2109.364258,12537520.0
8,2020-08-31,2174.000000,2174.000000,2060.350098,2080.699951,2074.028076,28648125.0
9,2020-09-01,2100.000000,2122.399902,2062.399902,2087.250000,2080.556885,17821397.0


In [ ]:
df.rename(columns=lambda x: x.lower(),inplace=True)
df.head(2)

,date,open,high,low,close,adj close,volume
0,2020-08-19,2141.0,2154.000000,2121.350098,2131.550049,2124.715088,15731396.0
1,2020-08-20,2120.0,2123.899902,2088.000000,2097.050049,2090.325684,10401212.0


In [ ]:
df.dropna(axis=0,inplace=True)
df.sort_values(by='date',inplace=True)
df.isna().sum()

date         0
open         0
high         0
low          0
close        0
adj close    0
volume       0
dtype: int64

In [ ]:
print(len(df))
df.describe().T

249


,count,mean,std,min,25%,50%,75%,max
open,249.0,2.068015e+03,1.102353e+02,1.837000e+03,1.980000e+03,2.071000e+03,2.136450e+03,2.325000e+03
high,249.0,2.089614e+03,1.114348e+02,1.905000e+03,1.997200e+03,2.089800e+03,2.153550e+03,2.369350e+03
low,249.0,2.040813e+03,1.084226e+02,1.830000e+03,1.960000e+03,2.035100e+03,2.104250e+03,2.310550e+03
close,249.0,2.062776e+03,1.096568e+02,1.841950e+03,1.976100e+03,2.055700e+03,2.134250e+03,2.324550e+03
adj close,249.0,2.057476e+03,1.101086e+02,1.836044e+03,1.969763e+03,2.053250e+03,2.130447e+03,2.317096e+03
volume,249.0,1.169959e+07,7.600158e+06,3.099956e+06,6.821015e+06,9.591456e+06,1.414742e+07,6.475046e+07


In [ ]:
df['date']=pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249 entries, 0 to 249
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   date       249 non-null    datetime64[ns]
 1   open       249 non-null    float64       
 2   high       249 non-null    float64       
 3   low        249 non-null    float64       
 4   close      249 non-null    float64       
 5   adj close  249 non-null    float64       
 6   volume     249 non-null    float64       
dtypes: datetime64[ns](1), float64(6)
memory usage: 15.6 KB


In [ ]:
print("starting date: ",min(df['date']))
print("end date: ",max(df['date']))
print("no of days: ",max(df['date'])-min(df['date']))

starting date:  2020-08-19 00:00:00
end date:  2021-08-18 00:00:00
no of days:  364 days 00:00:00


In [ ]:
df.groupby(df.date.dt.strftime('%B'))[['open','close']].mean().sort_values(by='close')

,open,close
date,,
January,1968.000006,1957.662494
April,1963.384207,1961.278956
November,1985.213160,1964.847367
May,1967.050006,1972.582513
December,1980.765897,1978.338645
February,2004.052509,2011.252509
July,2099.985712,2091.300008
March,2103.830950,2097.033349
August,2117.304543,2112.245461


In [ ]:
df.head(2)

,date,open,high,low,close,adj close,volume
0,2020-08-19,2141.0,2154.000000,2121.350098,2131.550049,2124.715088,15731396.0
1,2020-08-20,2120.0,2123.899902,2088.000000,2097.050049,2090.325684,10401212.0


In [ ]:
monthwise_high=df.groupby(df.date.dt.strftime('%B'))[['high']].max()

monthwise_low=df.groupby(df.date.dt.strftime('%B'))[['low']].min()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Bar(
    x=monthwise_high.index,
    y=monthwise_high,
    name='Stock high Price',
    marker_color='rgb(0, 153, 204)'
))
fig.add_trace(go.Bar(
    x=monthwise_low.index,
    y=monthwise_low,
    name='Stock low Price',
    marker_color='rgb(255, 128, 0)'
))

fig.update_layout(barmode='group', 
                  title=' Monthwise High and Low stock price')
fig.show()

In [ ]:
closedf=df[['date','close']]
fig = px.line(closedf, x=closedf.date, y=closedf.close,labels={'date':'Date','close':'Close Stock'})
fig.update_traces(marker_line_width=2, opacity=0.6)
fig.update_layout(title_text='Stock close price chart', plot_bgcolor='white', font_size=15, font_color='black')
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

In [ ]:
mms=MinMaxScaler(feature_range=(0,1))
del closedf['date']
closedf= mms.fit_transform(np.array(closedf['close']).reshape(-1, 1))


In [ ]:
train_split=int(len(closedf)*0.80)
train_data=closedf[:train_split]
test_data=closedf[train_split:]
print("train_data: ", train_data.shape)
print("test_data: ", test_data.shape)
print(train_data[:5])

train_data:  (199, 1)
test_data:  (50, 1)
[[0.60008297]
 [0.52859521]
 [0.49709925]
 [0.52590136]
 [0.49761728]]


In [ ]:
# def create_dataset(dataset, time_step=1):
#     dataX, dataY = [], []
#     for i in range(len(dataset)-time_step-1):
#         a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
#         print(a)
#         dataX.append(a)
#         dataY.append(dataset[i + time_step, 0])
#         print(dataX,dataY)
#     return np.array(dataX), np.array(dataY)
def create_dataset(dataset,time_step=2):
    df_x,df_y=[],[]
    for i in range(len(dataset)-time_step-1):
      df_x.append(dataset[i:i+time_step,0])
      df_y.append(dataset[i+time_step,0])
      # print(df_x,df_y)
    return np.array(df_x),np.array(df_y)


In [ ]:
time_step = 15
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

In [ ]:
from sklearn.svm import SVR

svr_rbf = SVR(
    kernel= 'rbf', C= 1e2, gamma= 0.1)
svr_rbf.fit(X_train, y_train)

SVR(C=100.0, gamma=0.1)

In [ ]:
from sklearn.metrics import mean_squared_error,mean_squared_log_error
train_predict=svr_rbf.predict(X_train)
test_predict=svr_rbf.predict(X_test)

train_predict = mms.inverse_transform(train_predict.reshape(-1,1))
test_predict = mms.inverse_transform(test_predict.reshape(-1,1))

act_y_train=mms.inverse_transform(y_train.reshape(-1,1))
act_y_test=mms.inverse_transform(y_test.reshape(-1,1))


print("Train data prediction:", train_predict.shape)
print("Test data prediction:", test_predict.shape)
print("training accuracy: ",mean_squared_log_error(train_predict,act_y_train)*100)
print("test accuracy: ",mean_squared_log_error(test_predict,act_y_test)*100)

Train data prediction: (183, 1)
Test data prediction: (34, 1)
training accuracy:  0.02864111331871631
test accuracy:  0.027913880145427807


In [ ]:
print("Train data explained variance regression score:", explained_variance_score(act_y_train, train_predict))
print("Test data explained variance regression score:", explained_variance_score(act_y_test, test_predict))

Train data explained variance regression score: 0.9090904675552282
Test data explained variance regression score: -0.1252040457949284


In [ ]:
print("Train data R2 score:", r2_score(act_y_train, train_predict))
print("Test data R2 score:", r2_score(act_y_test, test_predict))

Train data R2 score: 0.9087422022473035
Test data R2 score: -0.21382895584747197


In [ ]:
look_back=time_step
trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
trainPredictPlot[look_back:len(train_predict)+look_back, :] = train_predict
print("Train predicted data: ", trainPredictPlot.shape)
# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
testPredictPlot[len(train_predict)+(look_back*2)+1:len(closedf)-1, :] = test_predict
print("Test predicted data: ", testPredictPlot.shape)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])

plotdf = pd.DataFrame({'date': df['date'],
                       'original_close': df['close'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

Train predicted data:  (249, 1)
Test predicted data:  (249, 1)


In [ ]:
x_input=test_data[len(test_data)-time_step:].reshape(1,-1)
temp_input=list(x_input)
print(temp_input)
temp_input=temp_input[0].tolist()

from numpy import array

lst_output=[]
n_steps=time_step
i=0
pred_days = 10
while(i<pred_days):
    
    if(len(temp_input)>time_step):
        
        x_input=np.array(temp_input[1:])
        print("{} day input {}".format(i,x_input))
        x_input=x_input.reshape(1,-1)
        
        yhat = svr_rbf.predict(x_input)
        #print("{} day output {}".format(i,yhat))
        temp_input.extend(yhat.tolist())
        temp_input=temp_input[1:]
       
        lst_output.extend(yhat.tolist())
        i=i+1
        
    else:
        yhat = svr_rbf.predict(x_input)
        
        temp_input.extend(yhat.tolist())
        lst_output.extend(yhat.tolist())
        
        i=i+1
        
print("Output of predicted next days: ", len(lst_output))

[array([0.43783673, 0.40064248, 0.47772483, 0.50932449, 0.54247823,
       0.60567756, 0.51201833, 0.48735993, 0.50994622, 0.57055541,
       0.55646497, 0.6292994 , 0.68700784, 0.66784083, 0.68524634])]
1 day input [0.40064248 0.47772483 0.50932449 0.54247823 0.60567756 0.51201833
 0.48735993 0.50994622 0.57055541 0.55646497 0.6292994  0.68700784
 0.66784083 0.68524634 0.76322915]
2 day input [0.47772483 0.50932449 0.54247823 0.60567756 0.51201833 0.48735993
 0.50994622 0.57055541 0.55646497 0.6292994  0.68700784 0.66784083
 0.68524634 0.76322915 0.81167061]
3 day input [0.50932449 0.54247823 0.60567756 0.51201833 0.48735993 0.50994622
 0.57055541 0.55646497 0.6292994  0.68700784 0.66784083 0.68524634
 0.76322915 0.81167061 0.79604744]
4 day input [0.54247823 0.60567756 0.51201833 0.48735993 0.50994622 0.57055541
 0.55646497 0.6292994  0.68700784 0.66784083 0.68524634 0.76322915
 0.81167061 0.79604744 0.74566957]
5 day input [0.60567756 0.51201833 0.48735993 0.50994622 0.57055541 0.55

In [ ]:
last_days=np.arange(1,time_step+1)
day_pred=np.arange(time_step+1,time_step+pred_days+1)
print(last_days)
print(day_pred)
temp_mat = np.empty((len(last_days)+pred_days+1,1))
temp_mat[:] = np.nan
temp_mat = temp_mat.reshape(1,-1).tolist()[0]

last_original_days_value = temp_mat
next_predicted_days_value = temp_mat

last_original_days_value[0:time_step+1] = mms.inverse_transform(closedf[len(closedf)-time_step:]).reshape(1,-1).tolist()[0]
next_predicted_days_value[time_step+1:] = mms.inverse_transform(np.array(lst_output).reshape(-1,1)).reshape(1,-1).tolist()[0]

new_pred_plot = pd.DataFrame({
    'last_original_days_value':last_original_days_value,
    'next_predicted_days_value':next_predicted_days_value
})

names = cycle(['Last 15 days close price','Predicted next 10 days close price'])

fig = px.line(new_pred_plot,x=new_pred_plot.index, y=[new_pred_plot['last_original_days_value'],
                                                      new_pred_plot['next_predicted_days_value']],
              labels={'value': 'Stock price','index': 'Timestamp'})
fig.update_layout(title_text='Compare last 15 days vs next 10 days',
                  plot_bgcolor='white', font_size=15, font_color='black',legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15]
[16 17 18 19 20 21 22 23 24 25]


In [ ]:
#RandomForestRegressor
from sklearn.ensemble import RandomForestRegressor
rfr=RandomForestRegressor(n_estimators=100)
rfr.fit(X_train,y_train)

RandomForestRegressor()

In [ ]:
y_pred_train=rfr.predict(X_train)
y_pred_test=rfr.predict(X_test)

In [ ]:
print("Train data explained variance regression score:", explained_variance_score(y_pred_train, train_predict))
print("Test data explained variance regression score:", explained_variance_score(y_pred_test, test_predict))

Train data explained variance regression score: -226346.11828763355
Test data explained variance regression score: -247254.57882441342


In [ ]:
# reshape input to be [samples, time steps, features] which is required for LSTM
X_train =X_train.reshape(X_train.shape[0],X_train.shape[1] , 1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1] , 1)

print("X_train: ", X_train.shape)
print("X_test: ", X_test.shape)

X_train:  (183, 15, 1)
X_test:  (34, 15, 1)


In [ ]:
#Using LSTM model
tf.keras.backend.clear_session()
model=Sequential()
model.add(LSTM(32,return_sequences=True,input_shape=(X_train.shape[1],1)))
model.add(LSTM(32,return_sequences=True))
model.add(LSTM(32))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 32)            4352      
                                                                 
 lstm_1 (LSTM)               (None, 15, 32)            8320      
                                                                 
 lstm_2 (LSTM)               (None, 32)                8320      
                                                                 
 dense (Dense)               (None, 1)                 33        
                                                                 
Total params: 21,025
Trainable params: 21,025
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=200,batch_size=5,verbose=1)

Epoch 1/200
37/37 [==============================] - 11s 62ms/step - loss: 0.0796 - val_loss: 0.0207
Epoch 2/200
37/37 [==============================] - 1s 21ms/step - loss: 0.0352 - val_loss: 0.0155
Epoch 3/200
37/37 [==============================] - 1s 20ms/step - loss: 0.0329 - val_loss: 0.0097
Epoch 4/200
37/37 [==============================] - 1s 20ms/step - loss: 0.0316 - val_loss: 0.0063
Epoch 5/200
37/37 [==============================] - 1s 23ms/step - loss: 0.0306 - val_loss: 0.0070
Epoch 6/200
37/37 [==============================] - 1s 23ms/step - loss: 0.0265 - val_loss: 0.0055
Epoch 7/200
37/37 [==============================] - 1s 22ms/step - loss: 0.0238 - val_loss: 0.0078
Epoch 8/200
37/37 [==============================] - 1s 21ms/step - loss: 0.0216 - val_loss: 0.0045
Epoch 9/200
37/37 [==============================] - 1s 20ms/step - loss: 0.0208 - val_loss: 0.0051
Epoch 10/200
37/37 [==============================] - 1s 19ms/step - loss: 0.0194 - val_loss: 0.018